In [ ]:
!pip install jellyfish

In [2]:
import glob
import jellyfish
import numpy as np
import pandas as pd

In [3]:
concepts = pd.read_csv('OpenAlex concepts in use (17 August 2022) - concepts.tsv', sep='\t')
concepts.head()

,openalex_id,display_name,normalized_name,level,wikidata_id,parent_display_names,parent_ids
0,https://openalex.org/c17744445,Political science,political science,0,https://www.wikidata.org/wiki/Q36442,NaN,NaN
1,https://openalex.org/c138885662,Philosophy,philosophy,0,https://www.wikidata.org/wiki/Q5891,NaN,NaN
2,https://openalex.org/c162324750,Economics,economics,0,https://www.wikidata.org/wiki/Q8134,NaN,NaN
3,https://openalex.org/c144133560,Business,business,0,https://www.wikidata.org/wiki/Q4830453,NaN,NaN
4,https://openalex.org/c15744967,Psychology,psychology,0,https://www.wikidata.org/wiki/Q9418,NaN,NaN


In [4]:
concepts_list = concepts['normalized_name'].values
print(len(concepts_list))
concepts_list = concepts_list[~pd.isnull(concepts_list)]
print(len(concepts_list))

65026
65025


In [5]:
files = glob.glob("csv/*.csv") + glob.glob('csv/others/*.csv')
print(files[:4])

['csv/scholars_African Studies.csv', 'csv/scholars_Anthropology.csv', 'csv/scholars_Archaeology.csv', 'csv/scholars_Art and Design Teachers.csv']


In [6]:
fields_github = dict()
for file in files:
    name = file.split('/')[-1]
    if 'scholars_' in name:
        name = name[9:]
    name = name[:-4]
    fields_github[file] = name

In [7]:
fields_github

{'csv/scholars_African Studies.csv': 'African Studies',
 'csv/scholars_Anthropology.csv': 'Anthropology',
 'csv/scholars_Archaeology.csv': 'Archaeology',
 'csv/scholars_Art and Design Teachers.csv': 'Art and Design Teachers',
 'csv/scholars_Asian Studies.csv': 'Asian Studies',
 'csv/scholars_Atmospheric science and air quality.csv': 'Atmospheric science and air quality',
 'csv/scholars_Biblical Studies.csv': 'Biblical Studies',
 'csv/scholars_Bioinformatics.csv': 'Bioinformatics',
 'csv/scholars_Biophysics.csv': 'Biophysics',
 'csv/scholars_Book History (BookHistodons).csv': 'Book History (BookHistodons)',
 'csv/scholars_Chemistry.csv': 'Chemistry',
 'csv/scholars_Communication and Media Studies.csv': 'Communication and Media Studies',
 'csv/scholars_Criminology.csv': 'Criminology',
 'csv/scholars_Crowdsourcing in Cultural Heritage Citizen Science.csv': 'Crowdsourcing in Cultural Heritage Citizen Science',
 'csv/scholars_Dendrochronology.csv': 'Dendrochronology',
 'csv/scholars_Digital

In [8]:
for k,v in fields_github.items():
    print(k)
    dists = []
    for current in concepts_list:
        dists.append(jellyfish.levenshtein_distance(v, current))
    idxs = np.argsort(dists)
    for idx in idxs[:3]:
        print(concepts_list[idx])
    print()

csv/scholars_African Studies.csv
african studies
american studies
asian studies

csv/scholars_Anthropology.csv
anthropology
neurotology
chronology

csv/scholars_Archaeology.csv
archaeology
eschatology
rheology

csv/scholars_Art and Design Teachers.csv
art and design
software design pattern
row and column spaces

csv/scholars_Asian Studies.csv
asian studies
russian studies
urban studies

csv/scholars_Atmospheric science and air quality.csv
atmospheric instability
atmospheric sciences
atmospheric electricity

csv/scholars_Biblical Studies.csv
biblical studies
policy studies
animal studies

csv/scholars_Bioinformatics.csv
bioinformatics
geoinformatics
informatics

csv/scholars_Biophysics.csv
biophysics
geophysics
dinophysis

csv/scholars_Book History (BookHistodons).csv
history of ideas
history of astronomy
history of religions

csv/scholars_Chemistry.csv
chemistry
chemist
geochemistry

csv/scholars_Communication and Media Studies.csv
communication studies
communication disorder
communica

In [9]:
content = open('concepts_unification.txt').read().split('\n\n')
mapconcepts = dict()
for row in content:
    temp = row.split('\n')
    temp2 = []
    for item in temp:
        temp2.append(item.split(" (")[0])

    mapconcepts[temp[0]] = tuple(temp2[1:])
    

In [10]:
mapconcepts

{'csv/scholars_African Studies.csv': ('african studies',),
 'csv/scholars_Anthropology.csv': ('anthropology',),
 'csv/scholars_Archaeology.csv': ('archaeology',),
 'csv/scholars_Art and Design Teachers.csv': ('art and design',),
 'csv/scholars_Asian Studies.csv': ('asian studies',),
 'csv/scholars_Atmospheric science and air quality.csv': ('atmospheric sciences',),
 'csv/scholars_Biblical Studies.csv': ('biblical studies',),
 'csv/scholars_Bioinformatics.csv': ('bioinformatics',),
 'csv/scholars_Biophysics.csv': ('biophysics',),
 'csv/scholars_Book History (BookHistodons).csv': ('coding',),
 'csv/scholars_Chemistry.csv': ('chemistry',),
 'csv/scholars_Communication and Media Studies.csv': ('communication studies',),
 'csv/scholars_Criminology.csv': ('criminology',),
 'csv/scholars_Crowdsourcing in Cultural Heritage Citizen Science.csv': ('crowdsourcing software development',),
 'csv/scholars_Dendrochronology.csv': ('dendrochronology',),
 'csv/scholars_Digital Humanities.csv': ('digital

In [15]:
import ast
import math
from collections import defaultdict
    
def level_0_concept(concepts, current):
    if type(current['parent_ids']) == str:
        a = []
        fields = current['parent_ids'].split(', ')
        for current_id in fields:
            a += level_0_concept(concepts, concepts[concepts['openalex_id'] == current_id.lower()].iloc[0])
        return a
    else:
        return [current['normalized_name']]

mapconcepts_0level = defaultdict(lambda: []) # 19 giant -> subtopics
for file, fields in mapconcepts.items():
    for field in fields:
        print(field)
        temp = level_0_concept(concepts, concepts[concepts['normalized_name'] == field].iloc[0])
        for level0 in temp:
            mapconcepts_0level[level0].append(field)


african studies
anthropology
archaeology
art and design
asian studies
atmospheric sciences
biblical studies
bioinformatics
biophysics
coding
chemistry
communication studies
criminology
crowdsourcing software development
dendrochronology
digital humanities
education
gender studies
genealogy
history of science and technology
history and philosophy of science
history
information security
islamic theology
journalism
law
literary genre
marketing research
medical software
medieval history
meteorology
neuroscience
nuclear fusion
open science
operations research
management science
philosophy
planetary science
plant science
political science
psychology
public policy
qualitative research
religious studies
science technology and society
semantic web
sociology
theology
astrophysics
biophysics
cheminformatics
computational chemistry
communication sciences
geographer
special collections
museology
data archive
health psychology
application programming interface
palaeography
planetary science
social w

In [17]:
map_csv = []
for k, v in mapconcepts_0level.items():
    v = set(v)
    for subtopic in v:
        map_csv.append((k, subtopic))

pd.DataFrame(map_csv).to_csv('topic2subtopic.csv')